# Ocorrências SSP/SE

Base de áudios da SSP: https://www.ssp.se.gov.br/Audios

---

## Dependências

In [ ]:
# Instalações das dependências
!pip install git+https://github.com/openai/whisper.git -q
!pip3 install torch torchvision torchaudio -q
!pip install gradio -q
!pip install openai -q
!pip install pdfkit -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.5 MB/s eta 0:00:00


In [ ]:
!apt-get install wkhtmltopdf -q

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libdaemon0 libevdev2 libgudev-1.0-0 libhyphen0
  libinput-bin libinput10 libjson-glib-1.0-0 libjson-glib-1.0-common
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5
  libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1
  libsoup2.4-common libudev1 libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 libxkbcommon-x1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gradio as gr
import whisper
import os
import io
from google.colab import files, userdata
from openai import OpenAI
# from docx import Document
from contextlib import redirect_stdout
from markdown import markdown
from pdfkit import from_string

## Código da aplicação

In [ ]:
# Configurar chave da API do GPT-4
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

# Carregar o modelo Whisper aqui, fora da função
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:41<00:00, 75.1MiB/s]


In [ ]:
# Função para transcrição usando Whisper (modelo carregado globalmente)
def whisper_get_text(path: str) -> str:
    try:
        audio = whisper.load_audio(path)
        result = model.transcribe(audio, language="pt")
        return str(result["text"])
    except Exception as e:
        print(f"Erro na transcrição: {e}\n")
        return ""

In [ ]:
# Função para extrair entidades nomeadas com GPT-4
def extract_entities(text):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Você é um sistema de extração de entidades nomeadas, especializado em ocorrências policiais, que classifica as entidades de acordo com o papel que desempenham no contexto da fala, como 'Delegacia', 'Endereço', 'Suspeito', 'Vítima', 'Policial', 'Cidade', etc."},
                # {"role": "user", "content": f"Extraia as entidades nomeadas do seguinte texto e classifique-as de acordo com o papel que desempenham no contexto da fala:\n\n{text}"}
                {"role": "user", "content": f"Extraia as entidades nomeadas do texto mais abaixo e classifique-as de acordo com o papel que desempenham no contexto da fala. \
                Quando houver descrição de características físicas, adornos e acessórios utilizados por suspeitos e vítimas, coloque dentro do papel Suspeito ou papel Vítima, juntamente com os nomes. \
                Quando houver objetos, evidências encontradas no local, coloque dentro do papel Evidências. \
                Organize a saída, agrupando pelo papel com o seguinte formato: \
                Papel 1: \
                - Entidade 1 \
                - Entidade 2 \
                - etc.. \
                Papel 2: \
                - Entidade 1 \
                - etc.. \
                etc. \
                \n\n{text}"}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Erro na extração de entidades: {e}\n")
        return ""

In [ ]:
# Função para processar o áudio, transcrever, extrair entidades e gerar PDF
def process_audio(audio_file):
    transcription = whisper_get_text(audio_file)
    transcription = transcription.replace('\n', '<br>')
    entities = extract_entities(transcription)

    transcription_md = "# Transcrição da ocorrência\n\n" + transcription
    entities_with_title = f"## Entidades Identificadas\n\n{entities}"  # Adiciona título
    html_report = markdown(transcription_md) + markdown(entities_with_title, extensions=['nl2br'])  # Usa entities_with_title

    options = {
        'quiet': '',
        'encoding': "UTF-8"
    }
    # Gerar PDF com wkhtmltopdf
    pdf_file = "/tmp/relatorio_ocorrencia.pdf"  # Caminho para salvar o PDF
    from_string(html_report, pdf_file, options=options)

    return transcription_md, entities_with_title, pdf_file

## Interface Gradio

In [ ]:
# Interface Gradio com gr.Blocks()
with gr.Blocks() as iface:
    gr.Markdown("# Relato de ocorrência SSP/SE")
    gr.Markdown("Carregue um arquivo de áudio para obter a transcrição e o relatório em PDF.")
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Arquivo de áudio")
    with gr.Row():
        with gr.Column():
            # gr.Markdown("## Transcrição da Ocorrência")  # Subtítulo para a transcrição
            transcription_output = gr.Markdown(label="Transcrição")
        with gr.Column():
            # gr.Markdown("## Entidades Identificadas")  # Subtítulo para as entidades
            entities_output = gr.Markdown(label="Entidades")
        pdf_output = gr.File(label="Relatório em PDF")

    audio_input.change(
        fn=process_audio,
        inputs=audio_input,
        outputs=[transcription_output, entities_output, pdf_output]
    )

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c58368befa0dcbdee4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
